In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


# 0. Setup env


In [6]:
!git clone --quiet https://github.com/tensorflow/models.git

fatal: destination path 'models' already exists and is not an empty directory.


In [7]:
cd /content/models

/content/models


In [8]:
# !git reset 57e07520 --hard # v1.13.0
!git reset f7e99c08 --hard #v1.4.0

HEAD is now at f7e99c08 update broken ssd models in the zoo along with notebook.


In [9]:
!apt-get install -qq protobuf-compiler python-tk

In [10]:
!pip install -q Cython contextlib2 pillow lxml matplotlib PyDrive

In [11]:
!pip install -q pycocotools

In [12]:
cd /content/models/research

/content/models/research


In [13]:
!protoc object_detection/protos/*.proto --python_out=.

In [14]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

# 1. Setup Paths

In [15]:
BASE_PATH = '/content/drive/MyDrive/Learning/Udacity/Self-DrivingCarsND/Projects/CarND-Capstone-Traffic-light-classifier/'
WORKSPACE_PATH = BASE_PATH + 'Tensorflow/workspace'
SCRIPTS_PATH = BASE_PATH +'Tensorflow/scripts'
APIMODEL_PATH = BASE_PATH + 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
# IMAGE_PATH = WORKSPACE_PATH+'/images'
IMAGE_PATH = WORKSPACE_PATH+'/images/simulator_dataset_rgb'
MODEL_PATH = WORKSPACE_PATH+'/models'
COLAB_MODEL_PATH = WORKSPACE_PATH +'/Colab_models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
# CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
# CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

# 2. Train Model

In [16]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [17]:
CUSTOM_MODEL_NAME = 'ssd_inception_carnd_tl'
SUB_MODEL_NAME = 'ssd_inception_v2_300'
# CUSTOM_MODEL_NAME = 'ssd_mobilenet_carnd_tl'
CONFIG_PATH = COLAB_MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/'+SUB_MODEL_NAME +'/pipeline.config'
CONFIG_PATH

'/content/drive/MyDrive/Learning/Udacity/Self-DrivingCarsND/Projects/CarND-Capstone-Traffic-light-classifier/Tensorflow/workspace/Colab_models/ssd_inception_carnd_tl/ssd_inception_v2_300/pipeline.config'

In [18]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'eval_config': num_examples: 29
 max_evals: 3,
 'eval_input_config': label_map_path: "Tensorflow/workspace/annotations/label_map.pbtxt"
 shuffle: false
 num_epochs: 1
 num_readers: 1
 tf_record_input_reader {
   input_path: "Tensorflow/workspace/annotations/test.record"
 },
 'model': ssd {
   num_classes: 4
   image_resizer {
     fixed_shape_resizer {
       height: 300
       width: 300
     }
   }
   feature_extractor {
     type: "ssd_inception_v2"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9997000098228455
         center: true
         scale: true
         epsilon: 0.0010000000474974513
         train: true
       }
     }
   }
   box_coder {


In [19]:
cd {BASE_PATH}

/content/drive/MyDrive/Learning/Udacity/Self-DrivingCarsND/Projects/CarND-Capstone-Traffic-light-classifier


In [20]:
!python train.py --logtostderr --train_dir={COLAB_MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/'+SUB_MODEL_NAME +'/train_dir'} --pipeline_config_path={CONFIG_PATH}

Streaming output truncated to the last 5000 lines.
INFO:tensorflow:global step 17516: loss = 1.6924 (0.473 sec/step)
I0429 23:18:49.854920 139636783146880 learning.py:507] global step 17516: loss = 1.6924 (0.473 sec/step)
INFO:tensorflow:global step 17517: loss = 2.0758 (0.471 sec/step)
I0429 23:18:50.327053 139636783146880 learning.py:507] global step 17517: loss = 2.0758 (0.471 sec/step)
INFO:tensorflow:global step 17518: loss = 1.3096 (0.466 sec/step)
I0429 23:18:50.794640 139636783146880 learning.py:507] global step 17518: loss = 1.3096 (0.466 sec/step)
INFO:tensorflow:global step 17519: loss = 1.3124 (0.481 sec/step)
I0429 23:18:51.277528 139636783146880 learning.py:507] global step 17519: loss = 1.3124 (0.481 sec/step)
INFO:tensorflow:global step 17520: loss = 2.0003 (0.461 sec/step)
I0429 23:18:51.739952 139636783146880 learning.py:507] global step 17520: loss = 2.0003 (0.461 sec/step)
INFO:tensorflow:global step 17521: loss = 2.4841 (0.451 sec/step)
I0429 23:18:52.193028 139636

# 3. Export trained model to an frozen graph

In [ ]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path={CONFIG_PATH} --trained_checkpoint_prefix {COLAB_MODEL_PATH+'/'+CUSTOM_MODEL_NAME + '/model.ckpt-20000'} --output_directory={COLAB_MODEL_PATH+'/'+CUSTOM_MODEL_NAME + '/frozen_graph'}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0428 16:01:08.106532 139905537648512 module_wrapper.py:139] From export_inference_graph.py:104: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0428 16:01:08.114637 139905537648512 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:335: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0428 16:01:08.115370 139905537648512 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:144: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

Instructions for updating:
Use `tf.cas

# 8. Test Trained Model

In [ ]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
# from object_detection.builders import model_builder
import sys
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
## Test model
PATH_TO_GRAPH = COLAB_MODEL_PATH+'/'+CUSTOM_MODEL_NAME + '/frozen_graph/frozen_inference_graph.pb' ## load SSD trained on carla simulator images
PATH_TO_LABELS = ANNOTATION_PATH + '/label_map.pbtxt'
NUM_CLASSES = 4

In [ ]:
detection_graph = load_graph(PATH_TO_GRAPH)
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'Red'}, 2: {'id': 2, 'name': 'Green'}, 3: {'id': 3, 'name': 'Yellow'}, 4: {'id': 4, 'name': 'Unknown'}}


In [ ]:
FIG_SIZE = (12, 8)
# PATH_TO_IMGS = IMAGE_PATH+'/train'
# PATH_TO_IMGS = IMAGE_PATH+'/Archive/imgs'
PATH_TO_IMGS = WORKSPACE_PATH+'/images/train'
# PATH_TO_IMGS = WORKSPACE_PATH+'/images/Archive/train'
TEST_IMGS = glob(os.path.join(PATH_TO_IMGS, r'*.png'))

# random.shuffle(TEST_IMGS)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for idx, img_path in enumerate(TEST_IMGS):
#             random.shuffle(TEST_IMGS)
            image = Image.open(img_path)
            image_np = load_image_into_numpy_array(image)
            image_expanded = np.expand_dims(image_np, axis=0)
            
            (boxes, scores, classes, num) = sess.run(
                [detect_boxes, detect_scores, detect_classes, num_detections],
                feed_dict={image_tensor: image_expanded})
            
            print('SCORES')
            print(scores[0])
            print('CLASSES')
            print(classes[0])
            print(img_path)
            
            viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np, 
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                line_thickness=8)
            plt.figure(figsize=FIG_SIZE)
            plt.imshow(image_np)
            plt.show()
            
#             if idx == 10:
#                 break

Output hidden; open in https://colab.research.google.com to view.